In [3]:
import simpy
import random

Set globals:<br/>
&nbsp; 1. Infinite customers possible if simulation process extended to infinity<br/>
&nbsp; 2. Model customer temperment - propensity to regene using patience expvariate<br/>
&nbsp; 3. Customer arrival is 2 every min (approx)

In [4]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 99999999  # Total number of customers
INTERVAL_CUSTOMERS = 2.0  # Generate new customers roughly every x time units (mins)
MIN_PATIENCE = 5  # Min. customer waiting time post booking
MAX_PATIENCE = 15  # Max. customer witing time post booking - as per service

Waiting Time - Time from booking or on bus-stop arrival to on bus identity verification<br/>
&nbsp; 1. Set-up environment for both service types<br/>
&nbsp; 2. Constrain bus arrivals in dynamic routing service to the service policy 

In [3]:
def source_on_demand(env, number, interval, counter):
    """Source generates commuters randomly"""
    for i in range(number):
        bus_arrival = random.uniform(5,15)
        c = customer(env, 'Customer_%02d' % (i+1), counter, time_in_stop = bus_arrival)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_stop):
    """Commuter arrives at stop, waits for bus and is served or leaves."""
    arrive = env.now
    print('%7.4f %s: Arrives at Bus Stop' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the bus or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            if time_in_stop == 12.0:
                tib = random.expovariate(1.0 / time_in_stop)
            else:
                tib = service_bound_exponential(1.0 / time_in_stop)
            yield env.timeout(tib)
            print('%7.4f %s: Boards Bus after waiting for %6.3f' % (env.now, name, tib))
        else:
            print('%7.4f %s: RENEGED(I am taking a Grab!!!)/Lost after %6.3f' % (env.now, name, wait))

def source_deterministic(env, number, interval, counter):
    """Source generates commuters randomly"""
    for i in range(number):
        bus_arrival = 12.0 #deterministic - fixed frequency
        c = customer(env, 'Customer_%02d' % (i+1), counter, time_in_stop = bus_arrival)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)
        
def service_bound_exponential(alpha):
    gen = random.expovariate(alpha)
    while gen > 15:
        gen = random.expovariate(alpha)
    return gen

<b>Part A: As per on demand routing service policy 5-15 mins</b>

In [144]:
# Setup and start the simulation
print('Bus Stop Simulation - On Demand')
#random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=3)
env.process(source_on_demand(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run(until = 30)

Bus Stop Simulation - On Demand
 0.0000 Customer_01: Arrives at Bus Stop
 0.2640 Customer_02: Arrives at Bus Stop
 0.5706 Customer_03: Arrives at Bus Stop
 0.7614 Customer_02: Boards Bus after waiting for  0.497
 0.8273 Customer_03: Boards Bus after waiting for  0.257
 1.3694 Customer_01: Boards Bus after waiting for  1.369
 3.2510 Customer_04: Arrives at Bus Stop
 5.1595 Customer_05: Arrives at Bus Stop
 5.5211 Customer_05: Boards Bus after waiting for  0.362
 6.2583 Customer_04: Boards Bus after waiting for  3.007
 8.0373 Customer_06: Arrives at Bus Stop
 9.8311 Customer_07: Arrives at Bus Stop
10.2976 Customer_06: Boards Bus after waiting for  2.260
10.4375 Customer_08: Arrives at Bus Stop
11.3635 Customer_09: Arrives at Bus Stop
12.7119 Customer_10: Arrives at Bus Stop
13.8790 Customer_07: Boards Bus after waiting for  4.048
14.1907 Customer_10: Boards Bus after waiting for  0.312
15.9895 Customer_09: Boards Bus after waiting for  4.626
18.3775 Customer_11: Arrives at Bus Stop
19.4

<b>Part B: As per deterministic - fixed frequency scheduling</b>

In [156]:
# Setup and start the simulation
print('Bus Stop Simulation - Deterministic')
#random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=3)
env.process(source_deterministic(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run(until = 30)

Bus Stop Simulation - Deterministic
 0.0000 Customer_01: Arrives at Bus Stop
 4.3543 Customer_02: Arrives at Bus Stop
 4.7402 Customer_03: Arrives at Bus Stop
 5.9708 Customer_04: Arrives at Bus Stop
 6.5430 Customer_05: Arrives at Bus Stop
 8.8839 Customer_03: Boards Bus after waiting for  4.144
 8.9415 Customer_06: Arrives at Bus Stop
11.3026 Customer_07: Arrives at Bus Stop
11.9283 Customer_05: RENEGED(I am taking a Grab!!!)/Lost after  5.385
13.2516 Customer_08: Arrives at Bus Stop
14.5460 Customer_04: Boards Bus after waiting for  5.662
15.4549 Customer_09: Arrives at Bus Stop
16.1379 Customer_10: Arrives at Bus Stop
16.3659 Customer_11: Arrives at Bus Stop
16.9644 Customer_12: Arrives at Bus Stop
18.4162 Customer_13: Arrives at Bus Stop
18.7026 Customer_07: RENEGED(I am taking a Grab!!!)/Lost after  7.400
21.4281 Customer_01: Boards Bus after waiting for 21.428
22.5438 Customer_14: Arrives at Bus Stop
22.8696 Customer_15: Arrives at Bus Stop
23.2620 Customer_16: Arrives at Bus St

<b> Some Observations from simulations:</b><br/>
&nbsp; &nbsp; 1. Avg. waiting time is lesser in case of the service (4.323 mins vs 8.193 mins)<br/>
&nbsp; &nbsp; 2. Number of customers that renege is lesser in case of service (2 vs 6)<br/>